In [34]:
import time
import time, psutil, os
start = time.time()
m0 = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [35]:
import re
import unidecode
import polars as pl
import pandas as pd
import numpy as np
from rapidfuzz import fuzz
from pathlib import Path

# Moradas de Histórico

In [36]:
import polars as pl

ficheiro = Path.home() / "Downloads" / "06.BNE" / "addressGeoLoc_export_20221006.csv"

df = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["MORADA", "CP", "LOCALIDADE", "LATITUDE", "LONGITUDE"],
    decimal_comma=True
)

pl.Config.set_tbl_formatting("UTF8_FULL")  # formato mais limpo
pl.Config.set_tbl_rows(5)                 # nº de linhas a mostrar
pl.Config.set_tbl_width_chars(120) 
pl.Config.set_tbl_cols("10_000")    



polars.config.Config

In [37]:
df.head()

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""RUA CESARIO VERDE LOTE 3 A DAS…","""2660""","""FRIELAS""","""38.852697456""","""-9.16971532999997"""
"""RUA DO ROXICO NR50""","""3865-110""","""FERMELA""","""40.709808191""","""-8.54878568099997"""
"""AVENIDA 13 DE MAIO N 536""","""3885-227""","""CORTEGACA OVR""","""40.94109138""","""-8.61864986999996"""
"""AV REINALDO SANTOS N 24 3 DTO""","""2675-673""","""ODIVELAS""","""38.7926955990001""","""-9.18881750599996"""
"""RUA DO BARREIRO N 547""","""4405-730""","""VILA NOVA DE GAIA""","""41.113905909""","""-8.63062512299996"""


In [38]:
df_filtrado = df.filter(
    df["CP"].str.slice(0, 2).is_in(["40"])
)

df_filtrado

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""","""41.15159""","""-8.614601"""
"""RUA DA FIRMEZA 482""","""4000-216""","""PORTO""","""41.1510188030001""","""-8.60600840199993"""
"""RUA DE CAMOES NO93 5OA""","""4000-144""","""PORTO""","""41.152648013""","""-8.61019293599998"""
…,…,…,…,…
"""R NOSSA SENHORA DE FATIMA 322""","""4050-426""","""PORTO""","""41.1594219950001""","""-8.62593197999996"""
"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""","""41.15445""","""-8.610141"""


In [39]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # converter para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # fórmula haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371000  # raio da Terra em metros
    return R * c


# Moradas Base Nacional de Endereços (BNE) 

In [40]:

ficheiro = Path.home() / "Downloads" / "06.BNE" / "df_Principal_f.csv"

df_BNE = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["ART_COD",	"ART_TIPO","ART_TITULO","ART_DESIG","PORTA_NUM","CP4","CP3","CPALF","LOCALIDADE","LONG_PORTA","LAT_PORTA"],
    decimal_comma=True
)

In [41]:
df_BNE


ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE,LONG_PORTA,LAT_PORTA
i64,str,str,str,i64,i64,i64,str,str,str,str
61213,"""Rua""",null,"""Abraos""",49,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",32,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",10,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
…,…,…,…,…,…,…,…,…,…,…
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""


In [42]:

# ==========================
# 2. Constantes
# ==========================
R = 6371000.0
p = np.pi / 180.0
limite_deg = 0.005  # ~500m

# ==========================
# 3. Função para limpar coordenadas
# ==========================
def clean_coord(col):
    # substitui vírgulas → ponto e converte para float
    return (
        pl.col(col)
          .cast(pl.Utf8)
          .str.replace(",", ".")
          .replace(["None","nan","NaN",""], None)
          .cast(pl.Float64)
    )

# ==========================
# 4. Preparar bases (lazy → eager)
# ==========================
df_portas = (
    df_filtrado
    .with_columns([
        pl.col("CP").str.slice(0, 2).alias("CP2"),
        clean_coord("LATITUDE").alias("LATITUDE"),
        clean_coord("LONGITUDE").alias("LONGITUDE")
    ])
    .drop_nulls(["LATITUDE", "LONGITUDE", "CP2"])
)

df_bne = (
    df_BNE
    .with_columns([
        pl.col("CP4").cast(pl.Utf8).str.slice(0, 2).alias("CP2"),
        clean_coord("LAT_PORTA").alias("LAT_PORTA"),
        clean_coord("LONG_PORTA").alias("LONG_PORTA")
    ])
    .filter(pl.col("CP2") == "40")  # <--- filtro coerente
    .drop_nulls(["LAT_PORTA", "LONG_PORTA", "CP2"])
)



In [43]:
df_portas


MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE,CP2
str,str,str,f64,f64,str
"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40"""
"""RUA DA FIRMEZA 482""","""4000-216""","""PORTO""",41.151019,-8.606008,"""40"""
"""RUA DE CAMOES NO93 5OA""","""4000-144""","""PORTO""",41.152648,-8.610193,"""40"""
…,…,…,…,…,…
"""R NOSSA SENHORA DE FATIMA 322""","""4050-426""","""PORTO""",41.159422,-8.625932,"""40"""
"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",41.15445,-8.610141,"""40"""


In [44]:
# ==========================
# 5. Porta BNE – k=10 vizinhos mais próximos (<= 500 m)
# ==========================
import numpy as np
import polars as pl
from sklearn.neighbors import BallTree

# -------------------------------------------------
# Configurações
# -------------------------------------------------
R = 6371000.0
RAIO_M = 500.0
K_NEIGHBORS = 500   

# -------------------------------------------------
# 5.1 Coordenadas
# -------------------------------------------------
coords_portas = np.radians(df_portas.select(["LATITUDE", "LONGITUDE"]).to_numpy())
coords_bne = np.radians(df_bne.select(["LAT_PORTA", "LONG_PORTA"]).to_numpy())

# -------------------------------------------------
# 5.2 Busca k vizinhos
# -------------------------------------------------
tree = BallTree(coords_bne, metric="haversine")
distancias, indices = tree.query(coords_portas, k=K_NEIGHBORS)

# Flatten com reshape
dist_m = (distancias * R).flatten()
idx_bne = indices.flatten().astype(np.uint32)

# Índice da porta (repetido K vezes)
idx_porta = np.repeat(np.arange(len(df_portas), dtype=np.uint32), K_NEIGHBORS)

# -------------------------------------------------
# 5.3 Criar DataFrame + filtro
# -------------------------------------------------
pairs = (
    pl.DataFrame({
        "idx_porta": idx_porta,
        "idx_bne":   idx_bne,
        "dist_m":    dist_m
    })
    .filter(pl.col("dist_m") <= RAIO_M)
    .join(df_portas.with_row_count("idx_porta"), on="idx_porta", how="left")
    .join(df_bne.with_row_count("idx_bne"),    on="idx_bne",    how="left", suffix="_BNE")
    .drop(["idx_porta", "idx_bne"])
)

# -------------------------------------------------
# 6. Resultado
# -------------------------------------------------
print(f"Total de matches (k={K_NEIGHBORS}): {len(pairs)}")
pairs

/var/folders/8m/hh_3__ks6m12ytdrpdgv3zxc0000gn/T/ipykernel_21170/2419991662.py:44: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .join(df_portas.with_row_count("idx_porta"), on="idx_porta", how="left")
/var/folders/8m/hh_3__ks6m12ytdrpdgv3zxc0000gn/T/ipykernel_21170/2419991662.py:45: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .join(df_bne.with_row_count("idx_bne"),    on="idx_bne",    how="left", suffix="_BNE")


Total de matches (k=500): 10845312


dist_m,MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE,CP2,ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE_BNE,LONG_PORTA,LAT_PORTA,CP2_BNE
f64,str,str,str,f64,f64,str,i64,str,str,str,i64,i64,i64,str,str,f64,f64,str
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40""",3991213,"""Rua""",null,"""Cames""",295,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40""",3991213,"""Rua""",null,"""Cames""",801,4000,140,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40""",9831213,"""Rua""",null,"""Gonalo Cristvo""",301,4000,267,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
494.958229,"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",41.15445,-8.610141,"""40""",3991213,"""Rua""",null,"""Cames""",235,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
494.958229,"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",41.15445,-8.610141,"""40""",3991213,"""Rua""",null,"""Cames""",247,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""


# SAE

In [45]:
# ==========================
# 1. Imports + Dicionários (EXATAMENTE IGUAIS)
# ==========================
import polars as pl

abreviaturas = {
    "cmdt": "comandante", "cmte": "comandante",
    "dr": "doutor", "dra": "doutora",
    "sr": "senhor", "sra": "senhora", "srª": "senhora",
    "eng": "engenheiro", "enga": "engenheira", "engª": "engenheira",
    "prof": "professor", "profa": "professora", "profª": "professora",
    "arq": "arquiteto", "arqa": "arquiteta", "arqª": "arquiteta",
    "cap": "capitao", "maj": "major", "gen": "general",
    "ten": "tenente", "alm": "almirante",
    "sta": "santa", "sto": "santo", "s": "sao", "sao": "sao",
    "ns": "nossa senhora", "n s": "nossa senhora",
    "nsr": "nosso senhor", "n sr": "nosso senhor",
    "dom": "dom", "d": "dom",
    "visc": "visconde", "cond": "conde", "marq": "marques", "bar": "barao"
}

mapa_logradouros = {
    "r": "rua", "rua": "rua", "rª": "rua",
    "av": "avenida", "avd": "avenida", "avda": "avenida", "avenida": "avenida",
    "tv": "travessa", "trs": "travessa", "trav": "travessa", "travessa": "travessa",
    "pc": "praca", "pç": "praca", "pr": "praca", "prç": "praca", "praca": "praca",
    "lg": "largo", "al": "alameda", "bq": "beco",
    "esc": "escadas", "estr": "estrada", "cm": "caminho",
    "cç": "calcada", "qt": "quinta", "br": "bairro",
    "rot": "rotunda", "via": "via", "urb": "urbanizacao",
    "lgd": "lugar", "campo": "campo", "terreiro": "terreiro"
}

# ==========================
# 2. extrair_partes() – 100% SEU, SÓ OTIMIZADO
# ==========================
def extrair_partes(df: pl.DataFrame) -> pl.DataFrame:
    # --- 1. Normalização base ---
    df = df.with_columns([
        pl.col("MORADA")
          .str.to_lowercase()
          .str.replace_all(r"[\.]", "")
          .str.replace_all(r"\s+", " ")
          .str.strip_chars()
          .alias("morada_norm")
    ])

    # --- 2. Extrair logradouro (exato como você) ---
    logradouro = pl.col("morada_norm").str.extract(
        r"^(r|rua|av|avenida|tv|trav|travessa|pc|pr|praca|estr|estrada|beco|al|alameda|lg|largo|calcada|rotunda)", 1
    )

    # --- 3. Mapear logradouro (seu dicionário) ---
    logradouro_norm = logradouro.replace(mapa_logradouros, default=logradouro)

    # --- 4. Nome da rua (exato como você) ---
    nome_rua = pl.col("morada_norm").str.extract(
        r"^(?:r|rua|av|avenida|tv|trav|travessa|pc|pr|praca|estr|estrada|beco|al|alameda|lg|largo|calcada|rotunda)\s+([a-zçãõéóà\s]+?)(?:\d|$)", 
        1
    ).str.strip_chars()

    # --- 5. Normalizar nome_rua (seu código) ---
    expr_nome = nome_rua
    for log in set(mapa_logradouros.values()):
        expr_nome = expr_nome.str.replace_all(rf"\b{log}\b", "")
    nome_rua_norm = (
        expr_nome
        .str.replace_all(r"\b(n|nr|nº|nro|numero|num)\b", "")
        .str.replace_all(r"\d+", "")
        .str.replace_all(r"\s+", " ")
        .str.strip_chars()
    )

    # --- 6. Expandir abreviaturas (seu dicionário) ---
    nome_rua_final = nome_rua_norm
    for k, v in abreviaturas.items():
        nome_rua_final = nome_rua_final.str.replace_all(rf"\b{k}\b", v)

    # --- 7. Número, andar, CP, localidade (seu código + CP do BNE) ---
    df = df.with_columns([
        pl.col("morada_norm").str.extract(r"\s(\d+[a-z]?)\b", 1).alias("numero"),
        pl.col("morada_norm").str.extract(r"(\d+º\s*[a-z]*|r/c)", 1).alias("andar"),
        # Prioriza CP da coluna, senão regex
        pl.coalesce([
            pl.col("CP").str.replace_all(r"\s+", ""),
            pl.col("morada_norm").str.extract(r"(\d{4}-\d{3})", 1)
        ]).alias("codigo_postal"),
        pl.when(pl.col("LOCALIDADE").is_not_null())
          .then(pl.col("LOCALIDADE").str.to_lowercase())
          .otherwise(pl.col("morada_norm").str.extract(r"([a-zçãõéó]+)$", 1))
          .str.replace_all(r"\s+", " ")
          .str.strip_chars()
          .alias("localidade")
    ])

    # --- 8. Normalizações finais (seu código) ---
    df = df.with_columns([
        logradouro_norm.alias("logradouro_norm"),
        nome_rua_final.alias("nome_rua_final"),
        pl.col("numero").str.replace_all(r"[^0-9a-z]", "").alias("numero_norm"),
        pl.col("andar").str.replace_all(r"\s+", "").alias("andar_norm"),
        pl.col("codigo_postal").str.replace_all(r"\s+", "").alias("codigo_postal_norm"),
        pl.col("localidade").alias("localidade_norm")
    ])

    return df.select([
        "idx_porta",
        "logradouro_norm", "nome_rua_final", "numero_norm", "andar_norm",
        "codigo_postal_norm", "localidade_norm"
    ])

In [46]:
# ==========================
# 3. Aplicar extrair_partes (BNE com morada reconstruída)
# ==========================

# 1. Garantir idx_porta
pairs = pairs.with_row_index("idx_porta")

# 2. Lado 1: Histórico (inalterado)
df1_parts = extrair_partes(
    pairs.select(["idx_porta", "MORADA", "LOCALIDADE", "CP"])
).rename({col: col + "_hist" for col in [
    "logradouro_norm", "nome_rua_final", "numero_norm", 
    "andar_norm", "codigo_postal_norm", "localidade_norm"
]})

# 3. Lado 2: BNE → RECONSTRUIR MORADA
df2_parts = extrair_partes(
    pairs.with_columns(
        pl.concat_str([
            pl.col("ART_TIPO"),
            pl.lit(" "),
            pl.col("ART_TITULO").fill_null(""),
            pl.when(pl.col("ART_TITULO").is_not_null()).then(pl.lit(" ")).otherwise(pl.lit("")),
            pl.col("ART_DESIG")
        ]).alias("MORADA"),
        pl.col("LOCALIDADE_BNE").alias("LOCALIDADE"),
        pl.concat_str([pl.col("CP4"), pl.lit("-"), pl.col("CP3")]).alias("CP")
    ).select(["idx_porta", "MORADA", "LOCALIDADE", "CP"])
).rename({col: col + "_bne" for col in [
    "logradouro_norm", "nome_rua_final", "numero_norm", 
    "andar_norm", "codigo_postal_norm", "localidade_norm"
]})

# 4. Juntar
pairs_enriched = pairs.join(df1_parts, on="idx_porta").join(df2_parts, on="idx_porta")

/var/folders/8m/hh_3__ks6m12ytdrpdgv3zxc0000gn/T/ipykernel_21170/3088640684.py:54: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  logradouro_norm = logradouro.replace(mapa_logradouros, default=logradouro)


In [47]:
# ==========================
# 5. Fuzzy Scoring + Melhor Match
# ==========================
from rapidfuzz import fuzz

# --- Função de score (ajuste pesos conforme necessidade) ---
def calcular_fuzzy_score(row):
    # 1. Rua (token sort → ignora ordem)
    s_rua = fuzz.token_sort_ratio(row["nome_rua_final_hist"] or "", row["nome_rua_final_bne"] or "") / 100.0

    # 2. Número (exato ou próximo)
    num_h = str(row["numero_norm_hist"] or "")
    num_b = str(row["numero_norm_bne"] or "")
    s_num = fuzz.ratio(num_h, num_b) / 100.0 if num_h and num_b else 0.0

    # 3. Andar (opcional)
    s_andar = fuzz.ratio(row["andar_norm_hist"] or "", row["andar_norm_bne"] or "") / 100.0

    # 4. Código Postal (exato = 1.0)
    s_cp = 1.0 if (row["codigo_postal_norm_hist"] and 
                   row["codigo_postal_norm_hist"] == row["codigo_postal_norm_bne"]) else 0.0

    # 5. Localidade (token set → ignora repetições)
    s_loc = fuzz.token_set_ratio(row["localidade_norm_hist"] or "", row["localidade_norm_bne"] or "") / 100.0

    # 6. Distância espacial (quanto mais perto, melhor)
    s_dist = max(0.0, 1.0 - (row["dist_m"] / 500.0))

    # --- PESOS (total = 1.0) ---
    return (
        0.40 * s_rua +    # rua é o mais importante
        0.20 * s_num +    # número crítico
        0.15 * s_cp +     # CP confirma
        0.10 * s_loc +    # localidade ajuda
        0.10 * s_andar +  # andar é bônus
        0.05 * s_dist     # distância como tie-breaker
    )

# --- Aplicar score ---
pairs_scored = pairs_enriched.with_columns(
    pl.struct(pairs_enriched.columns)
      .map_elements(calcular_fuzzy_score, return_dtype=pl.Float64)
      .alias("fuzzy_score")
)

# --- Rank por morada ---
pairs_ranked = pairs_scored.with_columns(
    pl.col("fuzzy_score")
      .rank(method="dense", descending=True)
      .over("idx_porta")
      .alias("rank_fuzzy")
)

In [51]:
# ==========================
# 6. Melhor match por morada + FILTRO >= 85% (SEGURO)
# ==========================

# 1. Melhor match (rank = 1)
best_per_morada = (
    pairs_ranked
    .filter(pl.col("rank_fuzzy") == 1)
    .drop(["idx_porta", "rank_fuzzy"])
)

# 2. Total de moradas com match
total_moradas_com_match = len(best_per_morada)
print(f"Moradas totais com match: {total_moradas_com_match}")

# 3. Filtrar score >= 0.85
final_matches = best_per_morada.filter(pl.col("fuzzy_score") >= 0.75)

# 4. Estatísticas (seguro contra vazio)
total_com_85 = len(final_matches)
percentual = (total_com_85 / total_moradas_com_match) * 100 if total_moradas_com_match > 0 else 0

# Score médio (com fallback)
score_medio = final_matches["fuzzy_score"].mean()
if score_medio is not None:
    print(f"Moradas com score >= 85%: {total_com_85} ({percentual:.1f}%)")
    print(f"Score médio (>=85%): {score_medio:.3f}")
else:
    print(f"Moradas com score >= 85%: {total_com_85} (0.0%)")
    print("Score médio (>=85%): N/A (nenhum match com score >= 85%)")

# 5. Resultado final
final_matches.sort("fuzzy_score", descending=True)

Moradas totais com match: 10845312
Moradas com score >= 85%: 12065 (0.1%)
Score médio (>=85%): 0.762


dist_m,MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE,CP2,ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE_BNE,LONG_PORTA,LAT_PORTA,CP2_BNE,logradouro_norm_hist,nome_rua_final_hist,numero_norm_hist,andar_norm_hist,codigo_postal_norm_hist,localidade_norm_hist,logradouro_norm_bne,nome_rua_final_bne,numero_norm_bne,andar_norm_bne,codigo_postal_norm_bne,localidade_norm_bne,fuzzy_score
f64,str,str,str,f64,f64,str,i64,str,str,str,i64,i64,i64,str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,f64
128.001963,"""ESCADAS DO CODECAL N20 3 ANDAR""","""4000-173""","""PORTO""",41.141149,-8.61009,"""40""",5571213,"""Escadas""",null,"""Codeal""",58,4000,173,"""PORTO""","""Porto""",-8.61,41.14,"""40""",null,null,"""3""",null,"""4000-173""","""porto""",null,null,null,null,"""4000-173""","""porto""",0.7872
128.001963,"""ESCADAS DO CODECAL N20 3 ANDAR""","""4000-173""","""PORTO""",41.141149,-8.61009,"""40""",5571213,"""Escadas""",null,"""Codeal""",48,4000,173,"""PORTO""","""Porto""",-8.61,41.14,"""40""",null,null,"""3""",null,"""4000-173""","""porto""",null,null,null,null,"""4000-173""","""porto""",0.7872
128.001963,"""ESCADAS DO CODECAL N20 3 ANDAR""","""4000-173""","""PORTO""",41.141149,-8.61009,"""40""",5571213,"""Escadas""",null,"""Codeal""",50,4000,173,"""PORTO""","""Porto""",-8.61,41.14,"""40""",null,null,"""3""",null,"""4000-173""","""porto""",null,null,null,null,"""4000-173""","""porto""",0.7872
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
133.669413,"""RUA CAMOES N 769 2 ANDAR""","""4000-149""","""PORTO""",41.158798,-8.610048,"""40""",3991213,"""Rua""",null,"""Cames""",895,4000,149,"""PORTO""","""Porto""",-8.61,41.16,"""40""","""rua""","""camoes""","""769""",null,"""4000-149""","""porto""","""rua""","""cames""",null,null,"""4000-149""","""porto""",0.750269
133.669413,"""RUA CAMOES N 769 2 ANDAR""","""4000-149""","""PORTO""",41.158798,-8.610048,"""40""",3991213,"""Rua""",null,"""Cames""",901,4000,149,"""PORTO""","""Porto""",-8.61,41.16,"""40""","""rua""","""camoes""","""769""",null,"""4000-149""","""porto""","""rua""","""cames""",null,null,"""4000-149""","""porto""",0.750269


In [49]:
m1 = psutil.Process(os.getpid()).memory_info().rss / 1024**2
print("Tempo Polars:", round(time.time() - start, 2), "segundos")
print("Memória Polars:", round(m1 - m0, 2), "MB")

Tempo Polars: 121.73 segundos
Memória Polars: 2457.53 MB
